In [99]:
import random
class Sentiment:
    NEGATIVE='NEGATIVE'
    NEUTRAL='NEUTRAL'
    POSITIVE='POSITIVE'
class Reviews:
    def __init__(self,text,score):
        self.text=text
        self.score=score
        self.sentiment=self.get_sentiment()
    def get_sentiment(self):
        if self.score<=2:
            return Sentiment.NEGATIVE
        elif self.score==3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE
        
class ReviewContainer:
    def __init__(self,reviews):
        self.reviews=reviews
    def get_text(self):
        return [x.text for x in self.reviews]
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    def evenly_distribute(self):
        negative=list(filter(lambda  x: x.sentiment==Sentiment.NEGATIVE,self.reviews))
        positive=list(filter(lambda  x: x.sentiment==Sentiment.POSITIVE,self.reviews))
        positive_shrunk=positive[:len(negative)]
        self.reviews=negative+positive_shrunk
        random.shuffle(self.reviews)
        print(self.reviews.count(Sentiment.NEUTRAL))
        

In [2]:
import json
file_name='Books_small_10000.json'

reviews=[]
with open(file_name) as f:
    for line in f:
        review=json.loads(line)
        reviews.append(Reviews(review['reviewText'],review['overall']))

In [3]:
reviews[5].sentiment

'POSITIVE'

In [4]:
reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

## Prep data

In [100]:
from sklearn.model_selection import train_test_split

training,test=train_test_split(reviews,test_size=0.333, random_state=42)


train_container=ReviewContainer(training)

test_container=ReviewContainer(test)


train_container.evenly_distribute()
test_container.evenly_distribute()
cont.evenly_distribute()

0
0
860


In [101]:
train_x=train_container.get_text()
train_y=train_container.get_sentiment()


test_x=test_container.get_text()
test_y=test_container.get_sentiment()


## Bag of words vectorization

In [123]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer =TfidfVectorizer()
train_x_vectors=vectorizer.fit_transform(train_x)
test_x_vectors=vectorizer.transform(test_x)
#print(train_x_vectors[0])



# Classification

### Linear SVM

In [124]:
from sklearn import svm

clf_svm=svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors,train_y)

clf_svm.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

### Decision tree

In [125]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_x_vectors,train_y)

clf.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

### Naive bayes

In [126]:
from sklearn.naive_bayes import GaussianNB
clf_naive = GaussianNB()
clf_naive=clf_naive.fit(train_x_vectors.toarray(),train_y)

clf_naive.predict(test_x_vectors[0].toarray())

array(['POSITIVE'], dtype='<U8')

### Logistic regression

In [127]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression(random_state=0,multi_class='auto').fit(train_x_vectors, train_y)
clf_log.predict(test_x_vectors[0])

C:\Users\Baraka\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array(['NEGATIVE'], dtype='<U8')

## Evaluation

In [128]:
print(clf_svm.score(test_x_vectors,test_y))
# print(clf.score(test_x_vectors,test_y))
# print(clf_naive.score(test_x_vectors.toarray(),test_y))
# print(clf_log.score(test_x_vectors,test_y))

0.8177570093457944


In [129]:
from sklearn.metrics import f1_score

print(f1_score(test_y,clf_svm.predict(test_x_vectors),average=None,
    labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))
# print(f1_score(test_y,clf.predict(test_x_vectors),average=None,
#     labels=[Sentiment.POSITIVE,Sentiment.NEUTRAL,Sentiment.NEGATIVE]))
# print(f1_score(test_y,clf_naive.predict(test_x_vectors.toarray()),average=None,
#     labels=[Sentiment.POSITIVE,Sentiment.NEUTRAL,Sentiment.NEGATIVE]))
# print(f1_score(test_y,clf_log.predict(test_x_vectors),average=None,
#     labels=[Sentiment.POSITIVE,Sentiment.NEUTRAL,Sentiment.NEGATIVE]))

[0.81775701 0.81775701]


In [130]:
test=['very good','enjoyed it','']
new_test=vectorizer.transform(test)

clf_svm.predict(new_test)

array(['POSITIVE', 'POSITIVE', 'POSITIVE'], dtype='<U8')

## Tuning your model

In [133]:
from sklearn.model_selection import GridSearchCV

parameters={'kernel':('linear','rbf'),'C':(1,4,8,16,32)}

svc=svm.SVC()

clf=GridSearchCV(svc,parameters,cv=5)

clf.fit(train_x_vectors,train_y)

C:\Users\Baraka\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Baraka\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Baraka\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Baraka\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureW

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [134]:
print(f1_score(test_y,clf.predict(test_x_vectors),average=None,
    labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))

[0.80465116 0.8028169 ]


In [153]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier

parameters={'kernel':('linear','rbf'),'C':(1,4,8,16,32)}

parameters={
     'penalty' : ['l1', 'l2'],
     'C' : np.logspace(-4, 4, 20),
     'solver' : ['liblinear']
    },
    
log=LogisticRegression()

clf_log=GridSearchCV(log,parameters,cv=5)

clf_log.fit(train_x_vectors,train_y)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=({'C': array([1.000000...
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04]),
       

In [154]:
print(f1_score(test_y,clf_log.predict(test_x_vectors),average=None,
    labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))

[0.8156682  0.81042654]


## Save the model


In [156]:
import pickle

with open('./trained_model/sentiment_classifier.pkl','wb') as f:
    pickle.dump(clf_log,f)

In [158]:
with open('./trained_model/sentiment_classifier.pkl','rb') as f:
    loaded_model=pickle.load(f)

In [159]:
loaded_model.predict(test_x_vectors)

array(['NEGATIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE',
       'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE',
       'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE',
       'NEGATIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE',
       'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE',
       'POSITIVE', 'NEGATIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE',
       'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE',
       'POSITIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE',
       'POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'NEGATIVE',
       'NEGATIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE',
       'NEGATIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE',
       'POSITIVE', 'POSITIVE', 'POSITIVE', 'NEGATIVE', 'POSITIVE',
       'NEGATIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE',
       'POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE',
       'NEGATIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITI